# **Project Name - Music Recommandation System**

##### **Project Type**    - Recommandation
##### **Contribution**    - Individual
##### **Created by -** Prashik Dhole

# ***Let's Begin !***

### Import Libraries

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset Loading

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/songdata.csv')

### Dataset First View

In [4]:
df.head(10)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ..."
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \nYou're enc..."
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun \nCouldn't sle...
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,I'm waitin' for you baby \nI'm sitting all al...


In [5]:
df.tail(10)

,artist,song,link,text
57640,Zebrahead,The Setup,/z/zebrahead/the+setup_10198494.html,Lie to me \nTell me that everything will be a...
57641,Ziggy Marley,Freedom Road,/z/ziggy+marley/freedom+road_20531174.html,"That's why I'm marching, yes, I'm marching, \..."
57642,Ziggy Marley,Friend,/z/ziggy+marley/friend_20673508.html,[Chorus] \nI wanna thank you for the things y...
57643,Ziggy Marley,G7,/z/ziggy+marley/g7_20531173.html,Seven richest countries in the world \nThem h...
57644,Ziggy Marley,Generation,/z/ziggy+marley/generation_20531171.html,Many generation have passed away \nFighting f...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \nmake yourself at home \ni'm a bit ...


### Dataset Rows & Columns count

In [6]:
df.shape

(57650, 4)

In [7]:
df.columns

Index(['artist', 'song', 'link', 'text'], dtype='object')

### Dataset Information

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [9]:
df.describe()

,artist,song,link,text
count,57650,57650,57650,57650
unique,643,44824,57650,57494
top,Donna Summer,Have Yourself A Merry Little Christmas,/a/abba/ahes+my+kind+of+girl_20598417.html,I just came back from a lovely trip along the ...
freq,191,35,1,6


#### Missing Values/Null Values

In [10]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

#### Duplicate Values

In [11]:
df[df.duplicated()]

,artist,song,link,text


### What did you know about your dataset?

1.Dataset contains 57650 no.of.rows and 4 no.of.columns

2.No duplicate value present in the dataset.

3.No Null value Present in the dataset.


**Then For Recommandation Model we are using only 5000 no. of rows**

In [12]:
new_df = df.sample(n=5000)

In [13]:
new_df.head(10)

,artist,song,link,text
28739,Dave Matthews Band,My Heart Is Yours,/d/dave+matthews+band/my+heart+is+yours_207458...,When you hold my hand time stands still \nWhe...
19176,Tears For Fears,Lord Of Karma,/t/tears+for+fears/lord+of+karma_20135552.html,There's been some knocking \nThere's been som...
34532,Grateful Dead,Louie Louie,/g/grateful+dead/louie+louie_20236916.html,"Louie, Louie, me gotta go. Louie, Louie, me go..."
56826,X,Soul Kitchen,/x/x/soul+kitchen_20533552.html,"Well, the clock says it's time to close now \..."
17563,Richard Marx,Keep Coming Back,/r/richard+marx/keep+coming+back_20116037.html,You know just how to hurt me \nAnd how to tak...
4816,Ellie Goulding,Mirror,/e/ellie+goulding/mirror_21069555.html,Is this our world? \n \nI look in the mirror...
13115,Miley Cyrus,Hands In The Air,/m/miley+cyrus/hands+in+the+air_21068312.html,Let me see your hands in the air \n \nI reme...
28122,Cliff Richard,Mean Streak,/c/cliff+richard/mean+streak_20260133.html,"Well, you stand there honey, with you hands on..."
6900,Glee,Lean On Me,/g/glee/lean+on+me_20862336.html,Um um um um \nUm um um um \nUm um um um um ...
46801,Olly Murs,"C'mon, C'mon",/o/olly+murs/cmon+cmon_21076029.html,"When you walked in the room, I thought I'd for..."


In [14]:
new_df.shape

(5000, 4)

In [15]:
new_df = new_df.drop(['link'],axis=1).reset_index(drop=True)

In [16]:
new_df.head(10)

,artist,song,text
0,Dave Matthews Band,My Heart Is Yours,When you hold my hand time stands still \nWhe...
1,Tears For Fears,Lord Of Karma,There's been some knocking \nThere's been som...
2,Grateful Dead,Louie Louie,"Louie, Louie, me gotta go. Louie, Louie, me go..."
3,X,Soul Kitchen,"Well, the clock says it's time to close now \..."
4,Richard Marx,Keep Coming Back,You know just how to hurt me \nAnd how to tak...
5,Ellie Goulding,Mirror,Is this our world? \n \nI look in the mirror...
6,Miley Cyrus,Hands In The Air,Let me see your hands in the air \n \nI reme...
7,Cliff Richard,Mean Streak,"Well, you stand there honey, with you hands on..."
8,Glee,Lean On Me,Um um um um \nUm um um um \nUm um um um um ...
9,Olly Murs,"C'mon, C'mon","When you walked in the room, I thought I'd for..."


In [18]:
new_df['song'][0]

'My Heart Is Yours'

In [19]:
new_df['text'][0]

"When you hold my hand time stands still  \nWhen the love in my pain fills my heart  \nIt's your love that heals  \n  \nWhen I'm feeling this feeling inside  \nI know that it's true  \nThat every single beat of my heart  \nIs beating for you  \n  \nMy heart is yours and my love is too  \nAnd I love you with all of my heart  \nWhich belongs to you  \n  \nWhen your stuck and cold  \nAnd loving moves in  \nAnd love is shining bright as the sun  \nGives my soul from a veil  \n  \nThat every little kiss from your lips  \nTakes my breath away  \nAnd it's right here  \nBy your side I will always stay  \n  \nMy heart is yours  \nIt's where you'll be  \nYour the only love of my life  \nYour everything to me  \n  \nMy heart is yours  \nIs where you'll be  \nYour the only love of my life  \nYour everything to me  \n  \nIf I love you with all of my heart  \nWhich belongs to you\n\n"

**Removing Stopwords and Lower Casing:**

Removing stop words and lowercasing words are common pre-processing steps in natural language processing (NLP) tasks.

Stop words are words that are commonly used in a language but do not convey much meaning on their own, such as "a," "an," "the," and "is." These words can add noise to the data and can sometimes affect the performance of NLP models, so they are often removed as a pre-processing step.

Lowercasing words is the process of converting all the words in a text to lowercase. This is a common pre-processing step in NLP tasks, as it can be useful for a few reasons:

Case differences can be ignored: By lowercasing the words, you can treat words with different capitalization as the same word, which can be useful in tasks such as information retrieval or text classification where case differences are not important.
Vocabulary size is reduced: Lowercasing the words can also reduce the size of the vocabulary, which can make it easier to work with larger texts or texts in languages with a high number of inflected forms.

In [20]:
new_df['text'] = new_df['text'].str.lower().replace(r'[\w\s]','').replace(r'\n','',regex=True)

In [23]:
new_df['text'][0]

"when you hold my hand time stands still  when the love in my pain fills my heart  it's your love that heals    when i'm feeling this feeling inside  i know that it's true  that every single beat of my heart  is beating for you    my heart is yours and my love is too  and i love you with all of my heart  which belongs to you    when your stuck and cold  and loving moves in  and love is shining bright as the sun  gives my soul from a veil    that every little kiss from your lips  takes my breath away  and it's right here  by your side i will always stay    my heart is yours  it's where you'll be  your the only love of my life  your everything to me    my heart is yours  is where you'll be  your the only love of my life  your everything to me    if i love you with all of my heart  which belongs to you"

**Stemming**

Used SnowballStemmer to generate a meaningful word out of corpus of words.

Stemming is the process of reducing a word to its base or root form. This is a common pre-processing step in natural language processing (NLP) tasks, as it allows you to treat different inflected forms of a word as the same word, which can be useful for tasks like information retrieval or text classification.

For example, the words "run," "runs," "ran," and "running" are all different inflected forms of the same word "run," and a stemmer can reduce them all to the base form "run."

In [38]:
stemmer = SnowballStemmer("english")

def stemming(text):
  text = [stemmer.stem(word) for word in text.split()]

  return " ".join(text)

In [39]:
new_df['text'] = new_df['text'].apply(stemming)

In [40]:
new_df['text']

0       when you hold my hand time stand still when th...
1       there been some knock there been some heavi kn...
2       louie, louie, me gotta go. louie, louie, me go...
3       well, the clock say it time to close now i gue...
4       you know just how to hurt me and how to take a...
                              ...                        
4995    baby, won't you come and tell me tell me what ...
4996    from the highest throne to the earth below you...
4997    you say you want a revolut well, you know we a...
4998    i'm not as crazi as i use to be some of my dev...
4999    i can still rememb when all i had was time a t...
Name: text, Length: 5000, dtype: object

**Text Vectorization**

Word/Text vectorization is the process of representing words as numerical vectors. This is important in NLP tasks because most machine learning models expect numerical input and cannot work with raw text data directly. Word vectorization allows you to input the words into a machine learning model in a way that preserve the meaning and context of the words. Word vectorization can also be used to measure the similarity between words using vector arithmetic.

In [42]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(new_df['text'])
similarity = cosine_similarity(matrix)

In [43]:
similarity[0]

array([1.        , 0.03749372, 0.01054382, ..., 0.04752696, 0.05742163,
       0.27556153])

In [45]:
def recommendation(song_df):
    idx = new_df[new_df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(new_df.iloc[m_id[0]].song)

    return songs

In [46]:
recommendation('Alma Mater')

['Say Goodbye',
 'Goodbye',
 'So Now Goodbye',
 'Leaving',
 'Goodbye Apathy',
 'Girl Goodbye',
 'Long Long Way To Go',
 'Alone Together',
 '22',
 'Seconds',
 'Get In',
 'Miss Me Blind',
 'Let Go',
 'Say Goodbye To Hollywood',
 'Remember Me',
 'Too Hard To Say Goodbye',
 "I'm Gonna Miss You",
 "I Couldn't Say Goodbye",
 'Sunflowers',
 'Have I Grown Used To Missing You']

In [47]:
recommendation('Say Goodbye')

['So Now Goodbye',
 'Girl Goodbye',
 'Long Long Way To Go',
 'Too Hard To Say Goodbye',
 'Goodbye',
 "I Couldn't Say Goodbye",
 'Goodbye',
 'Seconds',
 'Leaving',
 'Alma Mater',
 'Goodnight',
 'Leaves That Are Green',
 'Say Goodbye To Hollywood',
 'Sorry',
 'Just Walk Away',
 'Slightly Drunk',
 'Better Off Now',
 'A Painter Passing Through',
 'I Say Yeah',
 'Remember Me']